* Navam Shrivastav 2019A7PS0092H
* Sukrit Kumar 2019AAPS0231H
* Shrikrishna Lolla 2019AAPS0345H

In [ ]:
import os
import pandas as pd
import re
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from itertools import permutations
import pickle


# Class


In [ ]:
class Node:
    def __init__(self, doc_Id):
        self.doc_id = doc_Id
        self.next = None

class LinkedList:
    def __init__(self):
        self.head = None
        self.tail = None
        self.size = 0
    
    def insert(self,Node):
        if self.head == None:
            self.head = Node
            self.tail = Node
        else:
            #temp = self.head
            self.tail.next = Node
            self.tail = Node
            #while temp.next != None:
                #temp = temp.next
            #temp.next = Node
        self.size += 1
    def size(self):
        return self.size
    
    def find(self,doc_id):
        temp = self.head
        while temp != None:
            if temp.doc_id == doc_id:
                return temp
            temp = temp.next
        return None
    
    def remove(self,doc_id):
        temp = self.head
        prev = None
        while temp != None:
            if temp.doc_id == doc_id:
                if prev == None:
                    self.head = temp.next
                else:
                    prev.next = temp.next
                self.size -= 1
                return
            prev = temp
            temp = temp.next
        return None

class InvertedIndex:
    """ InvertedIndex data structure class with insert, search and find functions"""
    def __init__(self):
        """
        Initializes the inverted index data structure.
        index: A dictionary that maps words to a list of doc_ids
        voc_list: A list of all the unique words in the index
        voc_count: The number of words in the index
        """
        self.index = {}
        self.voc_list = []
        self.voc_count = 0
    
    def insert(self,word,doc_id):
        """
        Inserts a word into the inverted index and adds a doc_id to the list 
        Args: word, doc_id
        """
        if word not in self.index:
            self.index[word] = [doc_id]
            if word[-1] == "$":
              self.voc_list.append(word)
            self.voc_count += 1
        if doc_id not in self.index[word]:
          self.index[word].append(doc_id)
    def search(self,word):
      """
      Searches the inverted index for a word and returns the list of doc_ids
      \n used in case of wildcard queries
      \n Args: word
      """
      final_output = []
      word_list = []
      for w in self.index.keys():
        if w.startswith(word):
          word_list.append(get_main(w))
          final_output.extend( self.index[w] )
      return final_output,word_list
    def find(self,word):
        """
        Find if a word exists in the inverted index
        and returns the list of doc_ids otherwise None
        \n used in case of regular queries 
        \n Args: word
        """
        if word in self.index:
            return self.index[word]
        else:
            return None
    def getSize(self):
      """
      Returns the number of words in the inverted index
      """
      return len(self.index)

def get_main(word):
  """get the original word where $ is the last character,rotate till we reach state """
  while(word[-1] != '$'):
    word = rotateString(word,1)
  return word

# intersection of 2 linked lists
def intersect(list1, list2):
    temp1 = list1.head
    temp2 = list2.head
    list3 = LinkedList()
    while temp1 != None and temp2 != None:
        if temp1.doc_id == temp2.doc_id:
            list3.insert(Node(temp1.doc_id))
            temp1 = temp1.next
            temp2 = temp2.next
        elif temp1.doc_id < temp2.doc_id:
            temp1 = temp1.next
        else:
            temp2 = temp2.next
    return list3

In [ ]:
def convert(a,doc_id):
  """ 
  Another approach to just use ints instead of a full list to store doc ids in a binary form
  for example if we store 9 as the value, the binary value will be 
  00001001 which means the 1st and 4th docs have the word
  """
  l = 1<<doc_id
  return a|l


In [ ]:
bin(convert(2,3))

'0b1010'

# Preprocessing functions

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def tokenize(s):
    """ Tokenize the string using word_tokenize from nltk """
    return word_tokenize(s)

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')
def remove_stop_words(s):
    """ Use nltk english stopwords and remove such tokens from our list """
    stop_words = set(stopwords)
    filtered_list = [w for w in s if w not in stop_words]
    return filtered_list

In [ ]:
def stem_words(word_list):
    """ Use the snowball stemming """ 
    stemmer = SnowballStemmer("english")
    stemmed_words = []
    for word in word_list:
        stemmed_words.append(stemmer.stem(word))
    return stemmed_words

In [ ]:
lemmatizer = WordNetLemmatizer()
def lemmatize(s):
    """ Lemmatization using the WordNet lemmatizer """ 

    return lemmatizer.lemmatize(s)

In [ ]:
def edit_distance(s1, s2):
    """ Edit distance function using dynamic programming """
    m = len(s1)
    n = len(s2)
    dp = np.zeros((m+1, n+1))
    for i in range(m+1):
        dp[i][0] = i
    for j in range(n+1):
        dp[0][j] = j
    for i in range(1, m+1):
        for j in range(1, n+1):
            if s1[i-1] == s2[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = 1 + min(dp[i-1][j], dp[i][j-1], dp[i-1][j-1])
    return dp[m][n]

In [ ]:
def rotateString(s, n):
    """ rotate string to the left by n places """
    return s[n:] + s[:n]

In [ ]:
def permutation(w):
  """ use permuterm for wildcard queries """ 
  w = w + "$"
  l = []
  for i in range(0,len(w)):
    l.append(rotateString(w,i))
  return l

In [ ]:
permutation("hello")

['hello$', 'ello$h', 'llo$he', 'lo$hel', 'o$hell', '$hello']

# Preprocessing Files

In [ ]:
!unzip dataset.zip -d IR

Archive:  dataset.zip
  inflating: IR/much-ado-about-nothing_TXT_FolgerShakespeare.txt  
  inflating: IR/richard-iii_TXT_FolgerShakespeare.txt  
  inflating: IR/the-winters-tale_TXT_FolgerShakespeare.txt  
  inflating: IR/richard-ii_TXT_FolgerShakespeare.txt  
  inflating: IR/henry-vi-part-3_TXT_FolgerShakespeare.txt  
  inflating: IR/the-two-noble-kinsmen_TXT_FolgerShakespeare.txt  
  inflating: IR/venus-and-adonis_TXT_FolgerShakespeare.txt  
  inflating: IR/timon-of-athens_TXT_FolgerShakespeare.txt  
  inflating: IR/the-merchant-of-venice_TXT_FolgerShakespeare.txt  
  inflating: IR/loves-labors-lost_TXT_FolgerShakespeare.txt  
  inflating: IR/troilus-and-cressida_TXT_FolgerShakespeare.txt  
   creating: IR/__MACOSX/
  inflating: IR/__MACOSX/._troilus-and-cressida_TXT_FolgerShakespeare.txt  
  inflating: IR/a-midsummer-nights-dream_TXT_FolgerShakespeare.txt  
  inflating: IR/lucrece_TXT_FolgerShakespeare.txt  
  inflating: IR/henry-iv-part-1_TXT_FolgerShakespeare.txt  
  inflating: IR

In [ ]:
doc_count = 0 # no of documents 

In [ ]:
doc_id_name_map = {} # map of int value and doc name

In [ ]:
def preProcess(directory):
    """ Main function to generate the inverted index """ 
    i_index = InvertedIndex()
    files = os.listdir(directory) # get all the files in the directory
    files_list = []
    for file in files:
        if file.endswith(".txt"): # only store text files
            files_list.append(file)
            #print(file)
    print(f"Total no of files is {len(files_list)}")
    global doc_count,doc_id_name_map
    doc_count = len(files_list)
    for i in range(0, len(files_list)):
      # i is doc id
      with open(directory + "/" + files_list[i], 'r') as f:
        text = f.read()
        doc_id_name_map[i+1] = files_list[i]
        #text = re.sub(r'[^\w\s]', '', text)
        text = re.sub("[^a-zA-Z]",' ',text) # Consider only alphabets
        text = text.lower() # convert to lowercase
        #print(text)
        print(i+1,files_list[i])
        tokens = tokenize(text) # tokenize
        rm_data = remove_stop_words(tokens) # remove stopwords
        stemmed = stem_words(rm_data) # perform stemming

        #print(f"size of data to be added {len(set(stemmed))}")
        for w in stemmed:

          #i_index.insert(rm_data[j],i)
          #if i == 0:
            #print(rm_data[j])

          p = permutation(w) # find all the permutations of the word
          for pw in p:
            i_index.insert(pw,i+1) # insert the word into the index
        #processFile(tokens,i,i_index)
        #print(f"Size of index after addition {i_index.getSize()}")
    i_Index_save = open('output_index_final', 'ab')
    pickle.dump(i_index, i_Index_save) # Save the generated index for later use                     
    i_Index_save.close()
    return i_index


In [ ]:
index = preProcess("IR")

Total no of files is 42
1 king-john_TXT_FolgerShakespeare.txt
2 romeo-and-juliet_TXT_FolgerShakespeare.txt
3 richard-iii_TXT_FolgerShakespeare.txt
4 pericles_TXT_FolgerShakespeare.txt
5 the-tempest_TXT_FolgerShakespeare.txt
6 richard-ii_TXT_FolgerShakespeare.txt
7 macbeth_TXT_FolgerShakespeare.txt
8 hamlet_TXT_FolgerShakespeare.txt
9 henry-viii_TXT_FolgerShakespeare.txt
10 as-you-like-it_TXT_FolgerShakespeare.txt
11 the-winters-tale_TXT_FolgerShakespeare.txt
12 troilus-and-cressida_TXT_FolgerShakespeare.txt
13 the-merry-wives-of-windsor_TXT_FolgerShakespeare.txt
14 the-taming-of-the-shrew_TXT_FolgerShakespeare.txt
15 henry-vi-part-2_TXT_FolgerShakespeare.txt
16 venus-and-adonis_TXT_FolgerShakespeare.txt
17 the-merchant-of-venice_TXT_FolgerShakespeare.txt
18 twelfth-night_TXT_FolgerShakespeare.txt
19 henry-v_TXT_FolgerShakespeare.txt
20 antony-and-cleopatra_TXT_FolgerShakespeare.txt
21 measure-for-measure_TXT_FolgerShakespeare.txt
22 much-ado-about-nothing_TXT_FolgerShakespeare.txt
23 h

In [ ]:
print(index.voc_list)

['king$', 'john$', 'william$', 'shakespear$', 'edit$', 'barbara$', 'mowat$', 'paul$', 'werstin$', 'michael$', 'poston$', 'rebecca$', 'nile$', 'folger$', 'librari$', 'https$', 'edu$', 'work$', 'creat$', 'oct$', 'fdt$', 'version$', 'charact$', 'play$', 'england$', 'dominion$', 'assort$', 'continent$', 'territori$', 'queen$', 'eleanor$', 'mother$', 'widow$', 'henri$', 'ii$', 'blanch$', 'spain$', 'niec$', 'princ$', 'son$', 'constanc$', 'geoffrey$', 'elder$', 'brother$', 'arthur$', 'duke$', 'brittani$', 'philip$', 'franc$', 'loui$', 'dauphin$', 'austria$', 'also$', 'call$', 'limog$', 'chatillion$', 'ambassador$', 'count$', 'melun$', 'french$', 'herald$', 'cardin$', 'pandulph$', 'papal$', 'legat$', 'ladi$', 'faulconbridg$', 'bastard$', 'richard$', 'robert$', 'sir$', 'jame$', 'gurney$', 'servant$', 'hubert$', 'support$', 'english$', 'nobl$', 'earl$', 'salisburi$', 'pembrok$', 'essex$', 'lord$', 'bigot$', 'citizen$', 'angier$', 'peter$', 'pomfret$', 'prophet$', 'execution$', 'messeng$', 'sheri

In [ ]:
print(len(index.voc_list))

14635


# Read from pregenerated pickle file

In [ ]:
""" Function to load and read from pre-existing pickle file """ 
def read_pickle():
  filename = "output_index_final"
  with open(filename,"rb") as f:
    index = pickle.load(f)
    return index

In [ ]:
index = read_pickle()

# Spelling correction

In [ ]:
"""
Return the k (default 5) words to the input query 
runs on against all the query terms where difference in length is 2 
"""
def kClosest(q,index,k = 5):
    output = {}
    q = q + "$"
    l = len(q)
    for w in index.voc_list:
      if abs(l - len(w)) <= 2:
        d = edit_distance(q,w)
        output[w] = d
    s = sorted(output.items(), key = lambda kv : kv[1])
    return s[:k]

In [ ]:
"""
function to get the correct word for each query term,
if query term exists in index return q_t directly,
otherwise find k closest values and return and choose the first one
"""
def getCorrect(q_t,index):
  if index.find(q_t):
    return q_t
  q_ed = kClosest(q_t,index)
  for i in q_ed:
    if int(i[1]) == 0:
      return q_t
    elif int(i[1]) <= 5:
      return i[0]
    else:
      return q_t

In [ ]:
kClosest("juleit",index)

[('juliet$', 2.0),
 ('jule$', 2.0),
 ('jupit$', 2.0),
 ('juli$', 2.0),
 ('julio$', 2.0)]

In [ ]:
getCorrect("juliet$",index)

'juliet$'

# Boolean query

In [ ]:

def and_op(l1,l2):
  """
  perform the and intersection on the two lists
  """
  i = 0
  j = 0
  intersection = []
  while i < len(l1) and j < len(l2):
    if(l1[i] == l2[j]):
      intersection.append(l1[i])
      i += 1
      j += 1
    elif (l1[i] < l2[j]):
      i += 1
    else:
      j += 1
  return intersection

In [ ]:
def or_op(l1,l2):
  """
  perform the or operation on the two lists
  """
  for i in l2:
    if i not in l1:
      l1.append(i)
  return l1

In [ ]:
def not_oper(l,size):
  """
  negate the list and create a new list with all elements in space size 
  and not in l
  """
  l_not = []
  for i in range(1,size+1):
    if i not in l:
      l_not.append(i)
  return l_not

In [ ]:
def getFilename(ret_list):
  """
  get the file names corresponding to the final retirval list 
  """
  for ret in ret_list[0]:
    print(f"Found in doc no {ret}  : {doc_id_name_map[ret]}")
  if len(ret_list[0]) == 0:
    print("Not found")

In [ ]:
def retrieve():
  """
  Query retrival function
  take query input from user
  tokenize it
  then create a retrival list and perform operations.
  """
  query = input()
  query = tokenize(query)
  #query = remove_stop_words(query)
  query = stem_words(query)
  print(query)
  retrival_list = []
  connect = []
  wc = 0
  conn = ["and$","or$","not$"]
  for q in query:
    q =  q + "$"
    if q in conn:
      connect.append(q)
      continue
    # don't correct wildcard queries 
    if "*" in q:
      wc = 0
      while(q[-1] != '*'):
        q = rotateString(q,1)
      l,l_w = index.search(q[:-1])
      l = list(set(l))
      print(f"Words returned for wildcard query {q} is {l_w}")
    else:
      q = getCorrect(q,index)
      l = index.find(q)
    if l:
      print(f"Query is {q} and list of docs are {l}")
    else:
      l = []
      print("No result found")
    if len(connect) >= 1 and connect[-1] == "not$":
      temp_cp = not_oper(l.copy(),doc_count)
      retrival_list.append(temp_cp)
    else:
      retrival_list.append(l.copy())
  if len(retrival_list) > 1:
    i_1 = 0
    i_2 = 1
    i = 0
    for con in connect:
      #print(con)
      if con == "and$":
        comp = and_op(retrival_list[i_1],retrival_list[i_2])
        temp_cp = retrival_list[2:]
        retrival_list = [comp]
        retrival_list.extend(temp_cp)
      elif con == "or$":
        #print("trig",i)
        comp = or_op(retrival_list[i_1],retrival_list[i_2])
        #print("computed" , comp)
        temp_cp = retrival_list[2:]
        #print("temp_cp", temp_cp)
        #comp.extend(temp_cp)
        retrival_list = [comp]
        retrival_list.extend(temp_cp)
        i += 1
        #print("ret_list",retrival_list)
      elif con == "not$":
        continue
  if len(query) == 0:
    return []
  if wc == 0:
    getFilename(retrival_list)
  return retrival_list


In [ ]:
retrieve()